# Frame to Phoneme Classifier

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
drivepath_shallow = '/content/gdrive/MyDrive/DL_Group_Project/Dataset/Preprocessed_Data'

In [15]:
drivepath_spec = '/content/gdrive/MyDrive/DL_Group_Project/experiments/specialized_detectors/models'

In [4]:
!pip install tqdm

In [5]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os
from tqdm import tqdm
import time
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd

In [6]:
NUM_EPOCHS = 100
BATCH_SIZE = 64
HIDDEN_SIZE_shallow = 128
HIDDEN_SIZE_spec = 128
MODEL_VERSION = 1
LEARNING_RATE = 0.01
LOGISTIC_THRESHOLD = 0.5
OTHER_PHONEMES_PERCENT = 0.1

In [7]:
cuda = torch.cuda.is_available()
num_workers = 8 if cuda else 0
DEVICE = "cuda" if cuda else "cpu"
print("Cuda = "+str(cuda)+" with num_workers = "+str(num_workers))

Cuda = True with num_workers = 8


In [8]:
class PhonemesDataset(Dataset):
    
    def __init__(self, basepath, mode):
      phoneme_features = np.zeros((1, 40))  # eliminate this row
      phoneme_labels = np.zeros((1))  # eliminate this row

      with os.scandir(basepath) as entries:
        for entry in entries:
          if entry.is_file():
            if "features" in entry.name and mode in entry.name:
              phoneme_tag = entry.name.split("_")[0]

              features_filepath = entry.path
              labels_filepath = f"{basepath}/{phoneme_tag}_{mode}_labels.npy"

              other_phoneme_features = np.load(features_filepath, allow_pickle=True)
              other_phoneme_labels = np.load(labels_filepath, allow_pickle=True)
              
              # stack to phoneme features
              phoneme_features = np.concatenate((phoneme_features, other_phoneme_features))
              phoneme_labels = np.concatenate((phoneme_labels, other_phoneme_labels))
              
      self.X = phoneme_features[1:]
      self.Y = phoneme_labels[1:]

    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, index):
        x = torch.Tensor(self.X[index]).float()
        y = torch.as_tensor(self.Y[index]).long()
    
        return x,y

In [9]:
class SpecializedDataset(Dataset):
    
    def __init__(self, datapath, mode, task_name, phonemes_class_0, phonemes_class_1):
      """
      phonemes_class_0: list of phoneme names for class 0
      phonemes_class_1: list of phoneme names for class 1
      """
      complete_features = np.zeros((1, 40))  # eliminate this row
      complete_labels = np.zeros((1))  # eliminate this row

      # go through all files in datapath, check phoneme if class=0 or class=1
      # assign that label and discard silence frames
      class_0_phonemes_found = []
      class_1_phonemes_found = []
      with os.scandir(datapath) as entries:
        for entry in entries:
          if entry.is_file():
            if "features" in entry.name and mode in entry.name:
              phoneme_tag = entry.name.split("_")[0]

              # find phoneme in class_0 or class_1 list and assign label
              phoneme_class = None
              if phoneme_tag in phonemes_class_0:
                phoneme_class = 0
                class_0_phonemes_found.append(phoneme_tag)
              if phoneme_tag in phonemes_class_1:
                phoneme_class = 1
                class_1_phonemes_found.append(phoneme_tag)

              if phoneme_class is None:
                print(f"phoneme '{phoneme_tag}' not found on class 0 nor class 1 lists; skip") if DEBUG else None
                continue

              print(f"phoneme '{phoneme_tag}' is class: {phoneme_class}") if DEBUG else None

              features_filepath = entry.path
              labels_filepath = f"{datapath}/{phoneme_tag}_{mode}_labels.npy"

              phoneme_features = np.load(features_filepath, allow_pickle=True)
              phoneme_labels = np.load(labels_filepath, allow_pickle=True)
              print(f"{phoneme_tag} total features: {phoneme_features.shape}") if DEBUG else None
              print(f"{phoneme_tag} total labels: {phoneme_labels.shape}") if DEBUG else None

              # find frames where label != 0 (non-silence)
              non_zero_indexes = phoneme_labels.nonzero()
              phoneme_features = phoneme_features[non_zero_indexes]
              phoneme_labels = phoneme_labels[non_zero_indexes]
              print(f"{phoneme_tag} no-silence features: {phoneme_features.shape}") if DEBUG else None
              print(f"{phoneme_tag} no-silence labels: {phoneme_labels.shape}") if DEBUG else None

              phoneme_labels[:] = phoneme_class  # label=class

              # stack to phoneme features
              complete_features = np.concatenate((complete_features, phoneme_features))
              complete_labels = np.concatenate((complete_labels, phoneme_labels))
      
      self.X = complete_features[1:]
      self.Y = complete_labels[1:]
      print(f"[task={task_name}] {self.X.shape} features")
      print(f"[task={task_name}] {self.Y.shape} labels")

      if sorted(class_0_phonemes_found) != sorted(phonemes_class_0):
        raise Exception(f"class 0 phonemes found ({sorted(class_0_phonemes_found)}) != expected phonemes ({sorted(phonemes_class_0)})")
 
      if sorted(class_1_phonemes_found) != sorted(phonemes_class_1):
        raise Exception(f"class 1 phonemes found ({sorted(class_1_phonemes_found)}) != expected phonemes ({sorted(phonemes_class_1)})")
 

    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, index):
        x = torch.Tensor(self.X[index]).float()
        y = torch.as_tensor(self.Y[index]).float()
    
        return x,y

In [10]:
def make_dataloader(dataset, train, batch_size):
  if train:
    shuffle = True
    drop_last = True
  else:
    shuffle = False
    drop_last = False
    
  loader = DataLoader(dataset=dataset, batch_size=batch_size,
                      drop_last=drop_last, shuffle=shuffle,
                      pin_memory=True, num_workers=8)
  
  return loader

In [11]:
class PhonemeShallowDetector(nn.Module):
  
  def __init__(self, hidden_size, activation):
    super(PhonemeShallowDetector, self).__init__()
    
    self.linear_layer = nn.Linear(in_features=40, out_features=hidden_size)
    self.bn_layer = nn.BatchNorm1d(num_features=hidden_size)
    self.activation = activation
    self.output_layer = nn.Linear(in_features=hidden_size, out_features=1)
    self.sigmoid = nn.Sigmoid()
    seq_params = [
      self.linear_layer,
      self.bn_layer,
      self.activation,
      self.output_layer,
      self.sigmoid
    ]

    self.network = nn.Sequential(*seq_params)
    
  def forward(self, x):
    return self.network(x)

In [12]:
class SpecializedShallowDetector(nn.Module):
  
  def __init__(self, hidden_size, activation):
    super(SpecializedShallowDetector, self).__init__()
    
    self.linear_layer = nn.Linear(in_features=40, out_features=hidden_size)
    self.bn_layer = nn.BatchNorm1d(num_features=hidden_size)
    self.activation = activation
    self.output_layer = nn.Linear(in_features=hidden_size, out_features=1)
    self.sigmoid = nn.Sigmoid()
    seq_params = [
      self.linear_layer,
      self.bn_layer,
      self.activation,
      self.output_layer,
      self.sigmoid
    ]

    self.network = nn.Sequential(*seq_params)
    
  def forward(self, x):
    return self.network(x)

In [27]:
class FramePhonemeClassifierModel(nn.Module):
  
  def __init__(self, phoneme_mapper, specialized_mapper):
    super(FramePhonemeClassifierModel, self).__init__()
    
    self.phoneme_mapper = phoneme_mapper
    self.specialized_mapper = specialized_mapper

    # specialized detectors

    specialized_detectors = []
    for spec_idx, spec_type in enumerate(SPECIALIZED_TASKS):
      specialized_detector = SpecializedShallowDetector(hidden_size=HIDDEN_SIZE_spec, 
                                            activation=nn.LeakyReLU()).to(DEVICE)
      specialized_detectors.append(specialized_detector)
    
    self.specialized_detectors = nn.ModuleList(specialized_detectors)


    shallow_detectors = []
    # generate the PhonemeShallowDetectors 
    for phoneme_index, phoneme_tag in phoneme_mapper.items():
      shallow_detector = PhonemeShallowDetector(hidden_size=HIDDEN_SIZE_shallow, 
                                                activation=nn.LeakyReLU())
      shallow_detectors.append(shallow_detector)

    self.shallow_detectors = nn.ModuleList(shallow_detectors)

    self.linear_layer = nn.Linear(in_features=len(phoneme_mapper), out_features=len(phoneme_mapper))

    self.initialize_specialized_detectors()
    self.initialize_shallow_detectors()

  def initialize_shallow_detectors(self):
    # load weights from shallow detectors pre-trained models
    for phoneme_index, phoneme_tag in self.phoneme_mapper.items():
      phoneme_shallow_detector = self.shallow_detectors[phoneme_index]

      phoneme_model_path = f"{drivepath_shallow}/shallow_detectors/model_{phoneme_tag}_{MODEL_VERSION}_99"
      temp = torch.load(phoneme_model_path)
      phoneme_shallow_detector.load_state_dict(temp['model_state_dict'])
  
  def initialize_specialized_detectors(self):
    # load weights from shallow detectors pre-trained models
    for spec_idx, spec_type in self.phoneme_mapper.items():
      specialized_detector = self.specialized_detectors[spec_idx]

      spec_model_path = f"{drivepath_spec}/model_{spec_type}_{MODEL_VERSION}_29"
      temp = torch.load(spec_model_path)
      specialized_detector.load_state_dict(temp['model_state_dict'])
  
  def forward(self, x):
    
    # go through specialized
    
    shallow_outputs = []
    for phoneme_index, phoneme_tag in self.phoneme_mapper.items():
      phoneme_shallow_detector = self.shallow_detectors[phoneme_index]

      # run frame through shallow detector
      output = phoneme_shallow_detector(x)
      shallow_outputs.append(output.reshape(-1))

    # convert to torch tensor
    shallow_outputs = torch.vstack(shallow_outputs).T
    outputs = self.linear_layer(shallow_outputs)

    return outputs

In [16]:
class FramePhonemeClassifier():

  def __init__(self, phoneme_mapper, specialized_mapper):

    train_data = PhonemesDataset(basepath=drivepath_shallow, mode="train")
    self.train_loader = make_dataloader(dataset=train_data, train=True, batch_size=BATCH_SIZE)
    print(f"train_data.shape: {train_data.X.shape}")

    dev_data = PhonemesDataset(basepath=drivepath_shallow, mode="dev")
    self.dev_loader = make_dataloader(dataset=dev_data, train=False, batch_size=BATCH_SIZE)
    print(f"dev_data.shape: {dev_data.X.shape}")
    
    self.model = FramePhonemeClassifierModel(phoneme_mapper, specialized_mapper).to(DEVICE)

    self.criterion = nn.CrossEntropyLoss()
    self.optimizer = torch.optim.SGD(self.model.parameters(), lr=LEARNING_RATE, momentum=0.9)
    self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, 'min')

    self.train_loss_per_epoch = []
    self.train_acc_per_epoch = []
    self.dev_loss_per_epoch = []
    self.dev_acc_per_epoch = []
  
  def save_model(self, epoch):
    model_epoch_path = "{}/complete_classifier/model_{}_{}".format(drivepath,
                                                                 MODEL_VERSION, 
                                                                 epoch)
    torch.save({
        'model_state_dict': self.model.state_dict(),
        'optimizer_state_dict': self.optimizer.state_dict(),
        'scheduler_state_dict': self.scheduler.state_dict(),
    }, model_epoch_path)
    # print('saved model: {}'.format(model_epoch_path))

  def train(self, epochs):
    # Run training and track with wandb
    total_batches = len(self.train_loader) * epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0

    for epoch in tqdm(range(epochs)):
        train_loss = 0.0
        start_time = time.time()
        total_predictions = 0
        correct_predictions = 0

        true_labels = []
        predictions = []
        for _, (features, targets) in enumerate(self.train_loader):
            batch_loss, outputs = self.train_batch(features, targets)
            train_loss += batch_loss

            example_ct += len(features)
            batch_ct += 1

            # check number of correct predictions
            output_classes = torch.argmax(outputs.log_softmax(1), dim=1).detach().cpu()  # convert to class labels
            total_predictions += len(output_classes)
            correct_predictions += torch.sum(targets == output_classes)

            true_labels += list(targets)
            predictions += list(output_classes)

        end_time = time.time()

        train_loss /= example_ct
        print(f"training loss: {train_loss}; time: {end_time - start_time}s")
        
        if (epoch + 1) % 10 == 0 or epoch == (epochs - 1):
          report = classification_report(true_labels, predictions, output_dict=True)
          df = pd.DataFrame(report).transpose()
          df.to_csv(f"{drivepath}/final_classsifier/reports_train_{MODEL_VERSION}_{epoch + 1}.csv", index=False)
          self.save_model(epoch)

        train_acc = (correct_predictions/total_predictions) * 100.0
        print(f"training accuracy: {train_acc}%")

        self.train_loss_per_epoch.append(train_loss)
        self.train_acc_per_epoch.append(train_acc)

        # evaluate model with validation data
        dev_loss, dev_acc = self.evaluate_model(epoch)
        
        self.dev_loss_per_epoch.append(dev_loss)
        self.dev_acc_per_epoch.append(dev_acc)

        # Step with the scheduler
        self.scheduler.step(dev_loss)
      
    # epoch completed, save model
    self.save_model(epoch)

  def train_batch(self, features, targets):
    features, targets = features.to(DEVICE), targets.to(DEVICE)
    targets = targets.reshape(-1, 1)

    self.optimizer.zero_grad()

    # Forward pass ➡
    outputs = self.model(features)
    loss = self.criterion(outputs, targets.reshape(-1))  # compare with target outputs
    # Backward pass ⬅
    loss.backward()
    # Step with optimizer
    self.optimizer.step()

    return loss.item(), outputs

  def evaluate_model(self, epoch):

    with torch.no_grad():
      self.model.eval()

      running_loss = 0.0
      total_predictions = 0.0
      correct_predictions = 0.0

      true_labels = []
      predictions = []

      example_ct = 0
      start_time = time.time()
      for batch_idx, (features, targets) in enumerate(self.dev_loader):
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
        targets = targets.reshape(-1, 1)

        example_ct += len(features)

        outputs = self.model(features)
        outputs = outputs.to(DEVICE)

        # check number of correct predictions
        output_classes = torch.argmax(outputs.log_softmax(1), dim=1)  # convert to class labels
        total_predictions += len(output_classes)
        correct_predictions += torch.sum(targets.reshape(-1) == output_classes)

        loss = self.criterion(outputs, targets.reshape(-1)).detach()
        running_loss += loss.item()

        true_labels += list(targets.detach().cpu())
        predictions += list(output_classes.detach().cpu())
      
      end_time = time.time()

      running_loss /= example_ct
      print(f"testing loss: {running_loss}; time: {end_time - start_time}s")
      acc = (correct_predictions/total_predictions) * 100.0
      print(f"testing accuracy: {acc}%")

      if (epoch + 1) % 10 == 0:
        report = classification_report(true_labels, predictions, output_dict=True)
        df = pd.DataFrame(report).transpose()
        df.to_csv(f"{drivepath}/final_classsifier/reports_dev_{MODEL_VERSION}_{epoch + 1}.csv", index=False)
  
      return running_loss, acc

# Train classifier

In [21]:
%cd /content/gdrive/MyDrive/DL_Group_Project/Dataset/Preprocessed_Data

/content/gdrive/.shortcut-targets-by-id/1qwJK2jyGMl2dPnVFe6JNZvrrG45HoonZ/DL_Group_Project/Dataset/Preprocessed_Data


In [23]:
from utilities import PHONEME_MAPPER
from utilities import SPECIALIZED_TASKS

In [ ]:
%cd /

/


In [ ]:
print(PHONEME_MAPPER)

{0: 'SIL', 1: 'AE', 2: 'AH', 3: 'AW', 4: 'AY', 5: 'B', 6: 'BIT', 7: 'D', 8: 'DH', 9: 'EE', 10: 'FF', 11: 'G', 12: 'HH', 13: 'IH', 14: 'II', 15: 'J', 16: 'K', 17: 'LL', 18: 'MM', 19: 'NN', 20: 'OH', 21: 'OO', 22: 'OW', 23: 'OY', 24: 'P', 25: 'RR', 26: 'SH', 27: 'SS', 28: 'T', 29: 'TH', 30: 'UE', 31: 'UH', 32: 'VV', 33: 'WW', 34: 'YY', 35: 'ZZ'}


In [24]:
print(SPECIALIZED_TASKS)

{'1_vowel_vs_consonant': {0: ['EE', 'IH', 'EH', 'AE', 'UH', 'ER', 'AH', 'AW', 'OO', 'UE'], 1: ['FF', 'HH', 'MM', 'NN', 'NG', 'RR', 'SS', 'SH', 'VV', 'WW', 'YY', 'ZZ']}, '3_highvowel_vs_lowvowel': {0: ['EE', 'IH', 'UE', 'OO'], 1: ['AE', 'AH', 'AW']}, '4_voiced_vs_unvoiced_fricatives': {0: ['DH', 'VV', 'ZZ'], 1: ['FF', 'SS', 'SH', 'TH']}, '5_ss_vs_zz': {0: ['SS'], 1: ['ZZ']}, '6_b_vs_p': {0: ['B'], 1: ['P']}, '7_dh_vs_th': {0: ['DH'], 1: ['TH']}, '8_ww_vs_yy': {0: ['WW'], 1: ['YY']}, '9_ee_vs_aw': {0: ['EE'], 1: ['AW']}, '10_ah_vs_aw': {0: ['AH'], 1: ['AW']}, '11_mm_vs_nn': {0: ['MM'], 1: ['NN']}}


In [ ]:
classifier = FramePhonemeClassifier(PHONEME_MAPPER, SPECIALIZED_TASKS)
classifier.train(epochs=NUM_EPOCHS)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train_data.shape: (49976, 40)
dev_data.shape: (10724, 40)








  0%|          | 0/100 [00:00<?, ?it/s]

training loss: 0.026293274411597314; time: 17.92014217376709s
training accuracy: 60.637020111083984%








  1%|          | 1/100 [00:19<32:19, 19.59s/it]

...

testing loss: 0.014187616531982746; time: 1.6525294780731201s
testing accuracy: 81.39686584472656%
training loss: 0.00025225303697491334; time: 16.699416399002075s
training accuracy: 88.0769271850586%








 36%|███▌      | 36/100 [11:03<19:32, 18.31s/it]

testing loss: 0.013388238724978367; time: 1.4855570793151855s
testing accuracy: 82.36665344238281%
training loss: 0.00024263251880867948; time: 17.00139880180359s
training accuracy: 88.21113586425781%








 37%|███▋      | 37/100 [11:22<19:17, 18.38s/it]

testing loss: 0.013360777038195737; time: 1.5115885734558105s
testing accuracy: 82.11488342285156%
